In [ ]:
##This is the NLP part of Analytics Project
#First 

In [1]:
pip install googlesearch-python 

  Using cached googlesearch_python-1.3.0-py3-none-any.whl.metadata (3.4 kB)
Using cached googlesearch_python-1.3.0-py3-none-any.whl (5.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 3.4 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 2.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import spacy
from transformers import pipeline 
import pandas as pd

names_sheet=pd.read_excel('GroupName_terrorists.xlsx')
names_sheet.dropna()
df=pd.DataFrame(names_sheet)

########################################################################################
# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

nlp = spacy.load('en_core_web_sm')

def get_first_google_link(gname):
    for result in search(gname, num_results=10):
        if 'wikipedia.org' in result:
            return result
        elif 'satp.org' in result:
            return result
    return None


##############################################################################################
def get_textual_content(group_name):
    # Get the first two sentences from the Wikipedia page
    first_link = get_first_google_link(group_name)
    if first_link:
        url = first_link
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p') 
            
            # Extract text from the paragraphs
            text = ' '.join([para.get_text() for para in paragraphs]) #getting full content of wikipedia
            
            # Split the text into sentences
            sentences = text.split('. ')
            
            # Return the first four sentences
            return '. '.join(sentences[:4]) + '.' if len(sentences) > 1 else None #if len(text)=1 it means text=name of grp
        else:
            return None
    else:
        return None
##############################################################################################


def classify_group(text):
    
    classes_labels = ["Religious Terrorist Group", "Separatist Terrorist", "Left-Wing Terrorist Group", "Right-Wing Terrorist Group", "Non terrorist Group"]
    result = classifier(text, classes_labels)
    return (result['labels'][0])

##############################################################################################

group_names = df['GNAME'].to_list()  
classified_list=[]
for group in group_names:
    content = get_textual_content(group)
    if content!= None:
        classified_list.append(classify_group(content))
    else: 
        classified_list.append('No information on Group')


length=len(df['GNAME'])-len(classified_list)

for i in range(length):
    classified_list.append('dummy')

df['Classification']=classified_list
print(df[['GNAME', 'Classification']])

df.to_excel('output_file.xlsx', index=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
Device set to use 0


                                             GNAME           Classification
0                         Achik Matgrik Army (AMA)  No information on Group
1                A'chik Matgrik Elite Force (AMEF)  No information on Group
2           Achik National Cooperative Army (ANCA)  No information on Group
3            Achik National Liberation Army (ANLA)  No information on Group
4          Achik National Volunteer Council (ANVC)     Separatist Terrorist
..                                             ...                      ...
312  Volunteers of Innocent People of Nagas (VIPN)  No information on Group
313              Yimchunger Liberation Front (YLF)      Non terrorist Group
314                                         Youths      Non terrorist Group
315                       Zeliangrong United Front  No information on Group
316                  Zomi Revolutionary Army (ZRA)      Non terrorist Group

[317 rows x 2 columns]


In [11]:
    from transformers import pipeline 
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    classes_labels = ["Finance","Oil and Gas", "Technology", "Agriculture"]
    result = classifier("I have earned huge returns by investing into bitcoin", classes_labels)
    print(result)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
Device set to use 0


{'sequence': 'I have earned huge returns by investing into bitcoin', 'labels': ['Finance', 'Technology', 'Agriculture', 'Oil and Gas'], 'scores': [0.6344659924507141, 0.3104734718799591, 0.03644862398505211, 0.018611909821629524]}
